In [1]:
#get json file from s3
from dcicutils import ff_utils
from functions.notebook_functions import *
import boto3
import json

s3 = boto3.resource('s3')

target_env = 'fourfront-webdev'
target_key = ff_utils.get_authentication_with_server({}, ff_env = target_env)

#target_addresses
target_health = ff_utils.get_metadata('/health', key = target_key)
target_raw = target_health['file_upload_bucket'] 
target_pf = target_health['processed_file_bucket'] 
target_att = target_health['blob_bucket'] 

source_raw = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
source_pf = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
source_att = 'elasticbeanstalk-fourfront-webdev-essentials-pack'


file_name = 'Soo_Webdev_essentials.json'
obj = s3.Object(bucket_name=source_att, key=file_name)
response =obj.get()
contents = response['Body'].read().decode('utf-8')
store = json.loads(contents)
for i in store:
    print(i, len(store[i]))

file_fastq 13
workflow_run_sbg 1
lab 9
file_reference 5
award 7
ontology 1
user 12
biosource 1
ontology_term 13
workflow 31
vendor 1
biosample 1
image 1
software 25
document 1
biosample_cell_culture 1
experiment_repliseq 1
organism 1
quality_metric_pairsqc 2
workflow_run_awsem 8
individual_human 1
file_processed 149
quality_metric_fastqc 8
protocol 3


In [6]:
#### This part should only run once!
# reverse lookup dictionary for schema names

schema_name = get_schema_names(target_key) 

rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=target_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            existing = ff_utils.get_metadata(an_item['uuid'], key=target_key)
            exists = True
        except:
            exists = False
        # skip the items that exists
        if exists and existing:
            remove_existing_items.append(an_item['uuid'])
            print("{} {} can not post item".format(obj_type, an_item['uuid']))
            continue
 
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = target_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(len(second_round_items[a_type]), 'items transfered to target')
    print()


user
12 items exist on source
user 986b362f-4eb6-4a9c-8173-3ab267307e3a can not post item
user 60938b2e-e120-4c4f-9ddb-001296021df7 can not post item
user 900c5bf0-5b3a-4d7c-a6e6-7918faa19a77 can not post item
user 83b5073a-069b-4162-9b30-6f42d5551e34 can not post item
user b2ceeab8-b15b-4fdd-bcf7-49aaca674a63 can not post item
user 1dbb8e49-38e4-4154-a97c-af8176a58c44 can not post item
user 92f90aed-7df1-4bd9-9e74-a472cb50d663 can not post item
user 57003524-6bea-44ba-98f0-42063bf5ee9b can not post item
user 5d5d5016-68a4-44c8-b79e-c31ada25629d can not post item
user 7c6184f0-3198-41d2-bb71-e7af0a9a74f0 can not post item
user 32c70fcf-f3af-4687-89e2-9bdd05cf460a can not post item
user 7b8aa7bc-eee0-4bc4-9892-1c46bc4552c5 can not post item
0 items transfered to target

award
7 items exist on source
award b0b9c607-f8b4-4f02-93f4-9895b461334b can not post item
award 6974f295-2c5d-48a1-b59d-99bc8d06bee7 can not post item
award 36a06537-7831-494d-b10d-3e9fea931021 can not post item
award a

8 items transfered to target



In [14]:
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        if 'flowcell_details' in an_item.keys():
            if not an_item.get('flowcell_details'):
                del an_item['flowcell_details']
                print(an_item['uuid'], an_item.get('flowcell_details'))
                continue
            else:
                for i in an_item['flowcell_details']:
                    if 'machine' not in i.keys():
                        an_item['flowcell_details'][0]['machine'] = 'unknown'
                        print(an_item['uuid'], an_item['flowcell_details'])
                        continue
        counter += 1
        if an_item.get('filename'):
            if an_item['filename'].endswith('_fastq.gz'):
                an_item['filename'] = an_item['filename'].replace('_fastq.gz', '.fastq.gz')
                print('updated filename')
        try:
            ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = target_key)
        except Exception as e:
            
            print(an_item)
            print(e)
            break
            

updated filename


In [15]:
# Round III - move attachments
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    for an_item in second_round_items[a_type]:
        if 'attachment' in an_item.keys():
            at_key = an_item['attachment']['blob_id']
            copy_source = {'Bucket': source_att, 'Key': at_key}
            try:
                s3.meta.client.copy(copy_source, target_att, at_key)
            except:
                print('Can not find attachment on source', an_item['uuid'])
                continue
            print('attachment copied')

attachment copied
attachment copied
attachment copied
attachment copied


In [18]:
# Round IV - move files
for a_type in my_types:
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
    else:
        continue
        
    for an_item in second_round_items[a_type]:
        # accumulate all keys from a file object to be uploaded
        files_to_upload = []
        file_resp = ff_utils.get_metadata(an_item['uuid'], key = target_key)
        # add extra file keys
        if file_resp.get('extra_files', []):
            for an_extra_file in file_resp['extra_files']:
                files_to_upload.append(an_extra_file['upload_key'])
        # add main file key
        files_to_upload.append(file_resp['upload_key'])
        
        for file_key in files_to_upload:
            copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
            
            try:
                s3.Object(target_file_bucket, file_key).load()
                print('file is already on target', file_key)
                print()
                continue
            except Exception as e:
                if e.response['Error']['Code'] == "404":
                    # The object does not exist.
                    pass
                else:
                    raise e
                    
            try:
                s3.meta.client.copy(copy_source, target_file_bucket, file_key)
            except:
                print('Can not find file on source', file_key, an_item['status'])
                continue
            print('file copied')

Can not find file on source 0048955c-7cb6-4e56-a4d8-56fad52f688a/4DNFIN232JJ2.fastq.gz deleted
file is already on target f4864029-a8ad-4bb8-93e7-5108f462ccaa/4DNFIRSRJH45.fastq.gz

file is already on target 1150b428-272b-4a0c-b3e6-4b405c148f7c/4DNFIVOZN511.fastq.gz

file is already on target 399898e9-b943-436c-93f5-edef293ab237/4DNFI25DCHPP.fastq.gz

file is already on target 0ddf1d70-91c9-483c-b836-c53af4946b20/4DNFI9JT48Q9.fastq.gz

Can not find file on source 0048955c-7cb6-4e56-a4d8-56fad52f688c/4DNFIN232JJ4.fastq.gz deleted
Can not find file on source 0048955c-7cb6-4e56-a4d8-56fad52f688d/4DNFIN232JJ5.fastq.gz deleted
Can not find file on source 46e82a90-49e5-4c33-afab-9ec90d65cca1/4DNFIO67AFHV.fastq.gz deleted
file is already on target a6ffb820-c600-4c32-9d99-249ed8029022/4DNFI2ZDNVFL.fastq.gz

file is already on target 38969bef-628a-41d6-bc74-c6b347c2e688/4DNFI43RP96B.fastq.gz

file is already on target 26e6a9a9-defe-46e4-bdc6-7af4675bd17a/4DNFI8KTY3QD.fastq.gz

Can not find file 

file copied
file copied
Can not find file on source 23125cc3-05da-481b-ab5f-0c58c2b42863/4DNFI9HPJJIB.pairs.gz deleted
file copied
Can not find file on source 6d1cb375-a5b1-4aaf-9e6e-5b066abbbd0b/4DNFI0DJCWQT.pairs.gz deleted
file copied
Can not find file on source 64bb56f0-f86b-4c2c-a790-450fe5b19930/4DNFICD6PHVJ.pairs.gz deleted
Can not find file on source c1e5b61a-a1d4-4d26-a1dd-554ecd4631c0/4DNFIGNMVV55.pairs.gz deleted
file copied
file copied
Can not find file on source 3581ab82-6670-453c-8ee6-cffee12be6f5/4DNFISTCZM9D.pairs.gz deleted
Can not find file on source ecabab05-3738-47fe-8b55-b08334463c43/4DNFINOFFTKV.bam deleted
Can not find file on source c1124f0d-043b-400f-90cb-e2b09b1ff9db/4DNFILYBGYVL.cool deleted
file copied
file copied
Can not find file on source 666768ca-a9cd-4415-8bf5-609ee69eb3f3/4DNFI86U73RR.pairs.gz deleted
file copied
file copied
file copied
file copied
file copied
Can not find file on source b8a0619d-f4dc-4b2b-b860-bbe99201b817/4DNFIZJU9SHE.pairs.gz delete